In [ ]:
from os import path

# Third-party
import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits, ascii
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

import gala.coordinates as gc

import dustmaps
from dustmaps.sfd import SFDQuery

import dustmaps.sfd
dustmaps.sfd.fetch()

---

### Polygon to get all data to check scanning law:

In [ ]:
(4*u.deg / (2*u.mas/u.yr)).to(u.Myr)

In [ ]:
poly = gc.GD1(phi1=[335, 335, 345, 345]*u.deg, 
              phi2=[-2, 2, 2, -2]*u.deg)
for c_ in poly.transform_to(coord.ICRS):
    print('{0.degree:.3f}, {1.degree:.3f},'.format(c_.ra, c_.dec))

SELECT *
FROM gaiadr2.gaia_source
WHERE CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
                       164.607, 45.703,
                       160.580, 48.621,
                       172.989, 55.072,
                       176.716, 51.744)) = 1

In [ ]:
scan = GaiaData('../data/gd1-gap-scanning-check.fits')

In [ ]:
scan_c = coord.SkyCoord(ra=scan.ra, dec=scan.dec).transform_to(gc.GD1)

In [ ]:
nan_mask = np.isnan(scan.pmra) | np.isnan(scan.pmdec)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))
ax.scatter(scan_c.phi1.degree[nan_mask], 
           scan_c.phi2.degree[nan_mask], 
           alpha=0.2, linewidth=0)
ax.set_title('nan pm')

fig, ax = plt.subplots(1, 1, figsize=(10, 3))
ax.scatter(scan_c.phi1.degree[~nan_mask], 
           scan_c.phi2.degree[~nan_mask], 
           alpha=0.08, linewidth=0)
ax.set_title('not nan pm')

fig, ax = plt.subplots(1, 1, figsize=(10, 3))
cs = ax.scatter(scan_c.phi1.degree, 
           scan_c.phi2.degree, 
           c=scan.visibility_periods_used,
           alpha=0.2, linewidth=0)
cb = fig.colorbar(cs)
cb.set_label('visibility periods')

---

In [ ]:
g = GaiaData(Table.read('gd1_tbl.fits'))
gd1_gal = coord.Galactic(l=g[g.parallax>0].skycoord.galactic.l,
                         b=g[g.parallax>0].skycoord.galactic.b)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.plot(gd1_gal.l.degree, gd1_gal.b.degree, marker='.', linestyle='none')
ax.set_xlim(300, 100)
ax.set_ylim(0, 80)

In [ ]:
l0, b0 = (170., 37.)
l = np.arange(l0 - 50., l0 + 50., 0.1)
b = np.arange(b0 - 50., b0 + 50., 0.1)
l, b = np.meshgrid(l, b)
coords = coord.SkyCoord(l*u.deg, b*u.deg,
                        distance=1.*u.kpc, frame='galactic')

In [ ]:
sfd = SFDQuery()
Av_sfd = 2.742 * sfd(coords)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 8))

ax.pcolormesh(l, b, 
   np.sqrt(Av_sfd)[::,::-1],
   vmin=0.,
   vmax=2.,
   cmap='binary'
)
ax.plot(gd1_gal.l.degree, gd1_gal.b.degree, 
        color='tab:orange', marker='.', linestyle='none')

ax.set_xlim(230, 110)
ax.set_ylim(0, 80)